# Lab 2.2.3 Solution: Object Detection Demo

**Module:** 2.2 - Computer Vision  
**Type:** Solution Notebook

---

This notebook contains solutions for object detection exercises using YOLO.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Exercise Solution: Detecting Specific Classes

This solution shows how to:
1. Download a custom image from URL
2. Filter detections for specific classes (e.g., only dogs and people)
3. Visualize the results

In [ ]:
def detect_custom_image(
    image_source: str,
    classes_to_detect: Optional[List[str]] = None,
    model_name: str = 'yolov8s.pt',
    confidence_threshold: float = 0.25
):
    """
    Detect objects in a custom image with optional class filtering.
    
    Args:
        image_source: Path or URL to image
        classes_to_detect: Optional list of class names to filter
                          (e.g., ['person', 'dog', 'car'])
        model_name: YOLO model to use
        confidence_threshold: Minimum confidence for detections
    
    Returns:
        Detection results
    """
    try:
        from ultralytics import YOLO
    except ImportError:
        print("Please install ultralytics: pip install ultralytics")
        return None
    
    # Load model
    model = YOLO(model_name)
    print(f"Loaded {model_name}")
    
    # Get class name to index mapping
    class_to_idx = {v: k for k, v in model.names.items()}
    
    # Filter classes if specified
    class_indices = None
    if classes_to_detect:
        class_indices = []
        for cls in classes_to_detect:
            if cls in class_to_idx:
                class_indices.append(class_to_idx[cls])
            else:
                print(f"Warning: '{cls}' not in model classes")
        
        if class_indices:
            print(f"Filtering for classes: {classes_to_detect}")
    
    # Run inference
    results = model(
        image_source,
        classes=class_indices,
        conf=confidence_threshold
    )
    
    # Display results
    result = results[0]
    annotated = result.plot()
    
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated[:, :, ::-1])  # BGR to RGB
    plt.title(f'Detected {len(result.boxes)} objects')
    plt.axis('off')
    plt.show()
    
    # Print detection details
    print(f"\nDetection Results:")
    print("="*50)
    for i, box in enumerate(result.boxes):
        cls_idx = int(box.cls)
        cls_name = model.names[cls_idx]
        conf = float(box.conf)
        coords = box.xyxy[0].tolist()
        print(f"{i+1}. {cls_name}: {conf:.2%} at [{coords[0]:.0f}, {coords[1]:.0f}, {coords[2]:.0f}, {coords[3]:.0f}]")
    
    return result


print("Detection function defined!")
print("\nUsage:")
print("  results = detect_custom_image('path/to/image.jpg')")
print("  results = detect_custom_image('image.jpg', classes_to_detect=['dog', 'person'])")

## Exercise Solution: Batch Processing

In [ ]:
def detect_batch(
    image_paths: List[str],
    model_name: str = 'yolov8s.pt',
    output_dir: str = './detections'
):
    """
    Process multiple images and save results.
    
    Args:
        image_paths: List of image paths
        model_name: YOLO model to use
        output_dir: Directory to save annotated images
    
    Returns:
        Summary statistics
    """
    try:
        from ultralytics import YOLO
    except ImportError:
        print("Please install ultralytics: pip install ultralytics")
        return None
    
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Load model
    model = YOLO(model_name)
    
    # Process images
    all_results = []
    class_counts = {}
    
    for img_path in image_paths:
        results = model(img_path)
        result = results[0]
        all_results.append(result)
        
        # Count classes
        for box in result.boxes:
            cls_name = model.names[int(box.cls)]
            class_counts[cls_name] = class_counts.get(cls_name, 0) + 1
        
        # Save annotated image
        output_file = output_path / f"detected_{Path(img_path).name}"
        result.save(str(output_file))
    
    # Summary
    print(f"\nBatch Processing Summary")
    print("="*50)
    print(f"Images processed: {len(image_paths)}")
    print(f"Total detections: {sum(class_counts.values())}")
    print(f"\nClass distribution:")
    for cls, count in sorted(class_counts.items(), key=lambda x: -x[1]):
        print(f"  {cls}: {count}")
    print(f"\nResults saved to: {output_dir}")
    
    return {
        'results': all_results,
        'class_counts': class_counts,
        'total_detections': sum(class_counts.values())
    }


print("Batch processing function defined!")

## Exercise Solution: Custom NMS Parameters

In [ ]:
def detect_with_custom_nms(
    image_source: str,
    iou_threshold: float = 0.45,
    conf_threshold: float = 0.25,
    max_detections: int = 300
):
    """
    Detect objects with custom NMS parameters.
    
    Args:
        image_source: Path to image
        iou_threshold: IoU threshold for NMS (higher = more overlapping boxes)
        conf_threshold: Confidence threshold (higher = fewer detections)
        max_detections: Maximum number of detections
    
    Returns:
        Detection results
    """
    try:
        from ultralytics import YOLO
    except ImportError:
        print("Please install ultralytics: pip install ultralytics")
        return None
    
    model = YOLO('yolov8s.pt')
    
    # Run with custom parameters
    results = model(
        image_source,
        conf=conf_threshold,
        iou=iou_threshold,
        max_det=max_detections
    )
    
    print(f"NMS Parameters:")
    print(f"  IoU threshold: {iou_threshold}")
    print(f"  Confidence threshold: {conf_threshold}")
    print(f"  Max detections: {max_detections}")
    print(f"\nDetections found: {len(results[0].boxes)}")
    
    return results[0]


print("Custom NMS detection function defined!")
print("\nNMS Parameter Effects:")
print("  - Higher IoU threshold: More overlapping boxes kept")
print("  - Lower IoU threshold: Fewer duplicates, may miss close objects")
print("  - Higher confidence: Fewer but more certain detections")

## Summary

Key concepts covered:

1. **Class Filtering**: Use `classes` parameter to detect only specific objects
2. **Batch Processing**: Efficiently process multiple images
3. **NMS Tuning**: Adjust IoU and confidence thresholds for your use case

YOLO provides a simple yet powerful interface for object detection tasks.

In [ ]:
# Cleanup
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Cleanup complete!")